**Aluno: Leonardo Ribeiro Schaedler**

# Atividade Spark

Considerando o dataset detalhado a seguir, extraia o conjunto de informações solicitadas.

### Dataset dados de COVID no Brasil

- Dados relativos a pacientes que realizaram exames de Covid19 no Brasil ate Marco 2020
- ~1.6GB
- 4.4M de instâncias


|  # | Nome do campo        	| Descrição                                     	|
|----|----------------------	|-----------------------------------------------	|
|  0 | id                   	| identificador                                 	|
|  1 | dataNotificacao      	| Data da notificação                           	|
|  2 | dataInicioSintomas   	| Data do inicio dos sintomas                   	|
|  3 | dataNascimento       	| Data de nascimento                            	|
|  4 | sintomas             	| Sintomas do paciente                          	|
|  5 | profissionalSaude    	| Relacionado a profissional de saúde           	|
|  6 | cbo                  	| Ocupação                                      	|
|  7 | condicoes            	| Condições do paciente                         	|
|  8 | estadoTeste          	| Estado do teste                               	|
|  9 | dataTeste            	| Data do teste                                 	|
| 10 | tipoTeste            	| Tipo de teste realizado                       	|
| 11 | resultadoTeste       	| Resultado do Teste                            	|
| 12 | paisOrigem           	| Pais de Origem do paciente                    	|
| 13 | sexo                 	| Sexo do paciente                              	|
| 14 | bairro               	| Bairro do paciente                            	|
| 15 | estado               	| Estado do paciente                            	|
| 16 | estadoIBGE              	| Estado do paciente IBGE                          	|
| 17 | municipio            	| Municipio do paciente                         	|
| 18 | municipioIBGE          	| Municipio do paciente                         	|
| 19 | cep            	        | CEP                                            	|
| 20 | origem               	| Origem do paciente                            	|
| 21 | cnes                 	| Código da unidade de saúde                    	|
| 22 | estadoNotificacao    	| Estado da notificação                         	|
| 23 | estadoNotificacaoIBGE    | Estado da notificação IBGE                      	|
| 24 | municipioNotificacao 	| Município da notificação                      	|
| 25 | municipioNotificacaoIBGE | Município da notificação IBGE                    	|
| 26 | numeroNotificacao    	| Número da notificação                         	|
| 27 | excluido             	| ID de exclusão                                	|
| 28 | validado             	| Local validação                               	|
| 29 | idade                	| Idade do paciente                             	|
| 30 | dataEncerramento     	| Data do encerramento da avaliação do paciente 	|
| 31 | evolucaoCaso         	| Evolução do caso do paciente                  	|
| 32 | classificacaoFinal   	| Avaliação final do caso                       	|

Informações a serem extraídas:

1. Quantidade de pacientes positivos para corona virus (resultadoTeste)
2. Quantidade de pacientes do sexo feminino positivos para corona virus (resultadoTeste)
3. Quantidade de pacientes de acordo com o sexo e o resultado do teste (resultadoTeste)
4. Sintomas mais comuns para casos positivos para corona virus
5. Sintomas mais comuns para casos não positivos para corona virus
6. Quantidade de casos positivos no Paraná
7. Municipio do Paraná com a maior quantidade de óbitos (evolucaoCaso)
8. Quantidade de municipios no Paraná com casos positivos
9. Quantidade de municipios no Paraná sem casos positivos
10. Estado com a maior taxa de falecimento de mulheres
11. Menor idade de mulher positiva para covid
12. Maior idade de mulher positiva para covid
13. Quantidade de casos positivos, em granularidade diária, ao longo do tempo
14. Quantidade de casos positivos, em granularidade semanal, ao longo do tempo
15. Quantidade de pacientes que faleceram (evolucaoCaso) separados pela idade
16. Taxa de óbito (evolucaoCaso) por idade
17. Idade média das mulheres positivas para covid
18. Município do Paraná com a maior quantidade de mulheres positivos para covid
19. Dia da semana com a maior quantidade de testes realizados
20. Dia da semana com maior quantidade de pacientes positivos para corona virus
21. Municipio, com mais de 500 testes, com a maior taxa de exames que não foram positivos para COVID (casos que foram testados porém não tiveram o resultadoTeste como Positivo)

**Dicas:**
- *Crie uma célula (Insert -> Insert Cell Below) para cada informação solicitada*

In [1]:
#instala pyspark no google colab
!pip install pyspark==3.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 25.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845494 sha256=bf3e6fc1ae53d380d4eb3f0e967170fb28781878925320642fed7ef818cf95e5
  Stored in directory: /root/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
# abre sessão no spark
import pyspark
conf = pyspark.SparkConf()
conf.setMaster('local[*]')
sc = pyspark.SparkContext.getOrCreate(conf)

In [3]:
#Efetua download do arquivo covid
!gdown 1EYMih3Ys1-DmaIZ9n0Sl5KAcRGqtpuj9

Downloading...
From: https://drive.google.com/uc?id=1EYMih3Ys1-DmaIZ9n0Sl5KAcRGqtpuj9
To: /content/covid.csv
100% 1.64G/1.64G [00:17<00:00, 96.2MB/s]


In [4]:
#carrega arquivo da COVID em um rdd
rdd = sc.textFile('file:////content/covid.csv')

In [5]:
rdd.map(lambda linha: linha.split(';')[15])\
  .filter(lambda estado: estado == 'PARANÁ')\
  .count()

41969

In [6]:
def pegaEstado(linha):
  campos = linha.split(';')
  return campos[15]

def isParana(estado):
  return estado == 'PARANÁ'

rdd.map(pegaEstado)\
  .filter(isParana)\
  .count()

41969

In [7]:
# qt de entradas para Paraná no bairro boa vista e cabral

def pegaCampos(linha):
  campos = linha.split(';')
  estado = campos[15]
  bairro = campos[14]
  return [estado, bairro]

rdd.map(pegaCampos)\
  .filter(lambda lista: lista[0] == 'PARANÁ')\
  .filter(lambda lista: lista[1] == 'Boa Vista' or lista[1] == 'Cabral')\
  .map(lambda lista: [lista[1], 1])\
  .reduceByKey(lambda x, y: x + y)\
  .take(5)

[('Boa Vista', 58), ('Cabral', 22)]

In [8]:
#informacao 1
#Quantidade de pacientes positivos para corona virus (resultadoTeste)

In [9]:
rdd.map(lambda linha: linha.split(';')[11])\
  .filter(lambda result: result == 'Positivo')\
  .count()

1230210

In [10]:
#informacao 2
#Quantidade de pacientes do sexo feminino positivos para corona virus (resultadoTeste)

In [11]:
def pegaCampos(linha):
  campos = linha.split(';')
  result = campos[11]
  sex = campos[13]
  return [result, sex]

rdd.map(pegaCampos)\
  .filter(lambda lista: lista[0] == 'Positivo')\
  .filter(lambda lista: lista[1] == 'Feminino')\
  .map(lambda lista: [lista[1], 1])\
  .reduceByKey(lambda x, y: x + y)\
  .take(1)

[('Feminino', 632899)]

In [12]:
#informacao 3
#Quantidade de pacientes de acordo com o sexo e o resultado do teste (resultadoTeste)

def pegaCampo(linha):
  campos = linha.split(';')
  sexo = campos[13]
  resultado = campos[11]
  return [sexo, resultado]

def geraChaveValor(entrada):
  return [entrada[0] + ' ' + entrada[1], 1]

rdd.map(pegaCampo)\
  .map(geraChaveValor)\
  .reduceByKey(lambda x, y: x + y)\
  .sortBy(lambda entrada: entrada[1], ascending=False)\
  .take(10)


[('Feminino Negativo', 1318173),
 ('Masculino Negativo', 1141559),
 ('Feminino Positivo', 632899),
 ('Masculino Positivo', 578177),
 ('Feminino null', 371661),
 ('Masculino null', 274303),
 ('Indefinido Negativo', 88174),
 ('Indefinido Positivo', 19117),
 ('null null', 453),
 ('null Negativo', 36)]

In [13]:
#informacao 4
#Sintomas mais comuns para casos positivos para corona virus


In [14]:
rdd.filter(lambda linha: linha.split(';')[11] == 'Positivo')\
  .map(lambda linha: linha.split(';')[4])\
  .flatMap(lambda s: [x.strip() for x in s.split(',')])\
  .map(lambda entrada: [entrada, 1])\
  .reduceByKey(lambda x,y: x + y)\
  .sortBy(lambda entrada: entrada[1], ascending=False)\
  .take(10)

[('Outros', 841588),
 ('Tosse', 576175),
 ('Febre', 462087),
 ('Dor de Garganta', 288681),
 ('Dispneia', 185306),
 ('Outros: Paciente assintomático', 24481),
 ('""', 14236),
 ('null', 10439),
 ('Dor De Garganta', 1476),
 ('', 528)]

In [15]:
#informacao 5
#Sintomas mais comuns para casos não positivos para corona virus


In [16]:
rdd.filter(lambda linha: linha.split(';')[11] != 'Positivo')\
  .map(lambda linha: linha.split(';')[4])\
  .flatMap(lambda s: [x.strip() for x in s.split(',')])\
  .map(lambda entrada: [entrada, 1])\
  .reduceByKey(lambda x,y: x + y)\
  .sortBy(lambda entrada: entrada[1], ascending=False)\
  .take(10)

[('Outros', 2089840),
 ('Tosse', 1124685),
 ('Febre', 752467),
 ('Dor de Garganta', 711525),
 ('Dispneia', 404983),
 ('Outros: Paciente assintomático', 260902),
 ('""', 42023),
 ('', 20507),
 ('Dor De Garganta', 16532),
 ('null', 14934)]

In [17]:
#informacao 6
#Quantidade de casos positivos no Paraná


In [18]:
rdd.filter(lambda linha: linha.split(';')[11] == 'Positivo' and linha.split(';')[15] == 'PARANÁ')\
  .count()

6183

In [19]:
#informacao 7
#Municipio do Paraná com a maior quantidade de óbitos (evolucaoCaso)

In [39]:
rdd.filter(lambda linha: linha.split(';')[15] == 'PARANÁ' and linha.split(';')[31] == "Óbito") \
  .map(lambda linha: (linha.split(';')[17], 1)) \
  .reduceByKey(lambda x, y: x + y) \
  .sortBy(lambda x: x[1], ascending=False) \
  .take(1)

[('Foz do Iguaçu', 25)]

In [21]:
#informacao 8
#Quantidade de municipios no Paraná com casos positivos


In [36]:
rdd.filter(lambda line: line.split(';')[11] == 'Positivo' and line.split(';')[15] == 'PARANÁ') \
  .map(lambda line: line.split(';')[17]) \
  .distinct() \
  .count()

296

In [22]:
#informacao 9
#Quantidade de municipios no Paraná sem casos positivos


In [37]:
rdd.filter(lambda line: line.split(';')[11] != 'Positivo' and line.split(';')[15] == 'PARANÁ') \
  .map(lambda line: line.split(';')[17]) \
  .distinct() \
  .count()

482

In [23]:
#informacao 10
#Estado com a maior taxa de falecimento de mulheres

In [112]:
def pegaCampos(linha):
    campos = linha.split(';')
    estado = campos[15]
    sexo = campos[13]
    evolucaoCaso = campos[31]
    obitoFeminino = sexo == 'Feminino' and 'Óbito' in evolucaoCaso
    return (estado, obitoFeminino)

rdd.map(pegaCampos) \
  .map(lambda x: (x[0], (1, int(x[1])))) \
  .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
  .map(lambda x: (x[0], (x[1][1] / x[1][0]) * 100 if x[1][0] > 0 else 0))\
  .sortBy(lambda entrada: entrada[1], ascending=False)\
  .first()



('BAHIA', 0.2837001068341712)

In [24]:
#informacao 11
#Menor idade de mulher positiva para covid

In [50]:
rdd.filter(lambda linha: linha.split(';')[11] == 'Positivo' \
  and linha.split(';')[13] == 'Feminino' \
  and 1 < int(linha.split(';')[29]) <= 116) \
  .map(lambda linha: int(linha.split(';')[29])) \
  .min()

# Optei pelo minimo de idade acima de 1 pois sem filtro estava retornando idade 0.

2

In [25]:
#informacao 12
#Maior idade de mulher positiva para covid

In [54]:
rdd.filter(lambda linha: linha.split(';')[11] == 'Positivo' \
  and linha.split(';')[13] == 'Feminino' \
  and 1 < int(linha.split(';')[29]) <= 116) \
  .map(lambda linha: int(linha.split(';')[29])) \
  .max()

# Optei pelo teto de idade 116 pois ser a maior idade já registrada no Brasil,
# sem filtro retornava idades impossíveis.

111

In [26]:
#informacao 13
#Quantidade de casos positivos, em granularidade diária, ao longo do tempo

In [61]:
def pegaCampos(linha):
  campos = linha.split(';')
  resultado = campos[11]
  data = campos[9]
  return [resultado, data]

rdd.map(pegaCampos)\
  .filter(lambda entrada: entrada[0] == 'Positivo')\
  .map(lambda entrada: [entrada[1][0:10], 1])\
  .reduceByKey(lambda x, y: x + y)\
  .sortBy(lambda entrada: entrada[0], ascending=True)\
  .take(10)

[('2020-01-01', 13),
 ('2020-01-02', 3),
 ('2020-01-06', 3),
 ('2020-01-07', 2),
 ('2020-01-08', 1),
 ('2020-01-09', 1),
 ('2020-01-10', 1),
 ('2020-01-13', 4),
 ('2020-01-15', 5),
 ('2020-01-16', 1)]

In [28]:
#informacao 14
#Quantidade de casos positivos, em granularidade semanal, ao longo do tempo

In [111]:
def pegaCampos(linha):
    campos = linha.split(';')
    resultado = campos[11]
    data = campos[9][0:10]
    ano, mes, dia = map(int, data.split('-'))
    numero_semana = (mes - 1) * 4 + (dia // 7 + 1)
    chave_semana = f"{ano}-W{numero_semana}"
    return [resultado, chave_semana]

indexed_rdd = rdd.zipWithIndex()
filtered_rdd = indexed_rdd.filter(lambda entrada: entrada[1] > 0).map(lambda entrada: entrada[0])

filtered_rdd.map(pegaCampos) \
  .filter(lambda entrada: entrada[0] == 'Positivo') \
  .map(lambda entrada: (entrada[1], 1)) \
  .reduceByKey(lambda x, y: x + y) \
  .sortBy(lambda entrada: entrada[0], ascending=True) \
  .take(10)

[('2020-W1', 19),
 ('2020-W10', 416),
 ('2020-W11', 2040),
 ('2020-W12', 4131),
 ('2020-W13', 11449),
 ('2020-W14', 11202),
 ('2020-W15', 16612),
 ('2020-W16', 20933),
 ('2020-W17', 49387),
 ('2020-W18', 36928)]

In [74]:
#informacao 15
#Quantidade de pacientes que faleceram (evolucaoCaso) separados pela idade

In [110]:
def pegaCampos(linha):
    campos = linha.split(';')
    idade = campos[29]
    evolucaoCaso = campos[31]
    return [idade, evolucaoCaso]

rdd.map(pegaCampos) \
  .filter(lambda entrada: 'Óbito' in entrada[1]) \
  .map(lambda entrada: (entrada[0], 1)) \
  .reduceByKey(lambda x, y: x + y) \
  .sortBy(lambda entrada: entrada[0], ascending=True) \
  .collect()

[('0', 271),
 ('1', 9),
 ('10', 3),
 ('100', 10),
 ('101', 7),
 ('102', 10),
 ('103', 4),
 ('104', 4),
 ('105', 2),
 ('107', 1),
 ('108', 2),
 ('110', 2),
 ('12', 4),
 ('13', 2),
 ('14', 2),
 ('15', 3),
 ('16', 12),
 ('17', 7),
 ('18', 10),
 ('188', 1),
 ('19', 14),
 ('2', 14),
 ('20', 14),
 ('21', 16),
 ('22', 21),
 ('23', 15),
 ('24', 15),
 ('25', 23),
 ('26', 33),
 ('27', 24),
 ('28', 28),
 ('29', 25),
 ('3', 7),
 ('30', 27),
 ('31', 19),
 ('32', 39),
 ('33', 43),
 ('34', 51),
 ('35', 43),
 ('36', 55),
 ('37', 65),
 ('38', 56),
 ('39', 66),
 ('4', 5),
 ('40', 54),
 ('41', 61),
 ('42', 74),
 ('43', 82),
 ('44', 75),
 ('45', 78),
 ('46', 91),
 ('47', 83),
 ('48', 85),
 ('49', 77),
 ('5', 3),
 ('50', 83),
 ('51', 88),
 ('52', 107),
 ('53', 105),
 ('54', 126),
 ('55', 140),
 ('56', 158),
 ('57', 124),
 ('58', 134),
 ('59', 161),
 ('6', 4),
 ('60', 153),
 ('61', 178),
 ('62', 169),
 ('63', 189),
 ('64', 173),
 ('65', 170),
 ('66', 193),
 ('67', 209),
 ('68', 205),
 ('69', 201),
 ('7', 6)

In [76]:
#informacao 16
#Taxa de óbito (evolucaoCaso) por idade

In [109]:
def pegaCampos(linha):
    campos = linha.split(';')
    idade = campos[29]
    evolucaoCaso = campos[31]
    obito = 'Óbito' in evolucaoCaso
    return (idade, obito)

rdd.map(pegaCampos) \
  .map(lambda entrada: (entrada[0], (1, int(entrada[1])))) \
  .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
  .map(lambda entrada: (entrada[0], (entrada[1][1] / entrada[1][0]) * 100 if entrada[1][0] > 0 else 0))\
  .collect()

[('937', 0.0),
 ('219', 0.0),
 ('336', 0.0),
 ('8', 0.01755001755001755),
 ('39', 0.05428747686613202),
 ('10', 0.02536354413256679),
 ('5952', 0.0),
 ('1051', 0.0),
 ('202', 0.0),
 ('824', 0.0),
 ('133', 0.0),
 ('826', 0.0),
 ('793', 0.0),
 ('942', 0.0),
 ('339', 0.0),
 ('118', 0.0),
 ('178', 0.0),
 ('41', 0.05449153148002572),
 ('20', 0.023931214851028188),
 ('97', 3.353658536585366),
 ('120', 0.0),
 ('352', 0.0),
 ('122', 0.0),
 ('5980', 0.0),
 ('935', 0.0),
 ('38', 0.04379379379379379),
 ('90', 3.303571428571429),
 ('191', 0.0),
 ('364', 0.0),
 ('247', 0.0),
 ('229', 0.0),
 ('3', 0.06013229104028864),
 ('125', 0.0),
 ('254', 0.0),
 ('965', 0.0),
 ('128', 0.0),
 ('35', 0.03702173089506492),
 ('2', 0.10562848951259998),
 ('359', 0.0),
 ('121', 0.0),
 ('1059', 0.0),
 ('1845', 0.0),
 ('962', 0.0),
 ('26', 0.03627689162003803),
 ('44', 0.07865510261869055),
 ('63', 0.5455805092084752),
 ('22', 0.029255245047504947),
 ('101', 4.216867469879518),
 ('136', 0.0),
 ('924', 0.0),
 ('959', 0.0

In [31]:
#informacao 17
#Idade média das mulheres positivas para covid

In [108]:
def pegaCampos(linha):
    campos = linha.split(';')
    sexo = campos[13]
    resultadoTeste = campos[11]
    idade = campos[29]
    return (sexo, resultadoTeste, idade)

indexed_rdd = rdd.zipWithIndex()
filtered_rdd = indexed_rdd.filter(lambda x: x[1] > 0).map(lambda x: x[0])

idade_positivos_femininos = filtered_rdd.map(pegaCampos) \
  .filter(lambda entrada: entrada[0] == 'Feminino' and entrada[1] == 'Positivo') \
  .map(lambda entrada: int(entrada[2]))

count = idade_positivos_femininos.count()
idade = idade_positivos_femininos.reduce(lambda x, y: x + y)
idade_media = idade / count if count > 0 else 0

print(f"Idade média das mulheres positivas para covid: {round(idade_media):.2f}")


Idade média das mulheres positivas para covid: 41.00


In [32]:
#informacao 18
#Município do Paraná com a maior quantidade de mulheres positivos para covid

In [107]:
def pegaCampos(linha):
    campos = linha.split(';')
    estado = campos[15]
    sexo = campos[13]
    resultadoTeste = campos[11]
    municipio = campos[17]
    return (estado, sexo, resultadoTeste, municipio)



# Filter for female positive cases in Paraná and then group by city
rdd.map(pegaCampos) \
  .filter(lambda entrada: entrada[0] == 'PARANÁ' and entrada[1] == 'Feminino' and entrada[2] == 'Positivo') \
  .map(lambda entrada: (entrada[3], 1)) \
  .reduceByKey(lambda x, y: x + y) \
  .sortBy(lambda entrada: entrada[1], ascending=False) \
  .first()


('Curitiba', 903)

In [33]:
#informacao 19
#Dia da semana com a maior quantidade de testes realizados

In [106]:
def pegaCampos(linha):
    campos = linha.split(';')
    dataTeste = campos[9][0:10]
    ano, mes, dia = map(int, dataTeste.split('-'))
    diaDaSemana = (ano + ((mes + 9) % 12) * 26 // 10 + dia + (ano // 4) - (ano // 100) + (ano // 400)) % 7
    dias = ["Domingo", "Segunda", "Terça", "Quarta", "Quinta", "Sexta", "Sábado"]
    return dias[diaDaSemana]


indexed_rdd = rdd.zipWithIndex()
filtered_rdd = indexed_rdd.filter(lambda entrada: entrada[1] > 0).map(lambda x: x[0])


filtered_rdd.map(lambda linha: (pegaCampos(linha), 1)) \
  .reduceByKey(lambda x, y: x + y) \
  .sortBy(lambda entrada: entrada[1], ascending=False) \
  .first()

('Sábado', 874481)

In [34]:
#informacao 20
#Dia da semana com maior quantidade de pacientes positivos para corona virus

In [105]:
def pegaCampos(linha):
    campos = linha.split(';')
    resultadoTeste = campos[11]
    dataTeste = campos[9][0:10]
    ano, mes, dia = map(int, dataTeste.split('-'))
    diaDaSemana = (ano + ((mes + 9) % 12) * 26 // 10 + dia + (ano // 4) - (ano // 100) + (ano // 400)) % 7
    dias = ["Domingo", "Segunda", "Terça", "Quarta", "Quinta", "Sexta", "Sábado"]
    return (dias[diaDaSemana], resultadoTeste)

indexed_rdd = rdd.zipWithIndex()
filtered_rdd = indexed_rdd.filter(lambda x: x[1] > 0).map(lambda x: x[0])

filtered_rdd.map(pegaCampos) \
  .filter(lambda entrada: entrada[1] == 'Positivo') \
  .map(lambda entrada: (entrada[0], 1)) \
  .reduceByKey(lambda x, y: x + y) \
  .sortBy(lambda entrada: entrada[1], ascending=False) \
  .first()

('Sábado', 240776)

In [35]:
#informacao 21
#Municipio, com mais de 500 testes, com a maior taxa de exames que não foram positivos para COVID (casos que foram testados porém não tiveram o resultadoTeste como Positivo)

In [104]:
def pegaCampos(linha):
    campos = linha.split(';')
    municipio = campos[17]
    testesNaoPositivos = campos[11] != 'Positivo'
    return (municipio, (1, int(testesNaoPositivos)))

rdd.map(pegaCampos) \
  .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
  .filter(lambda entrada: entrada[1][0] > 500) \
  .map(lambda entrada: (entrada[0], (entrada[1][1] / entrada[1][0]) * 100)) \
  .sortBy(lambda entrada: entrada[1], ascending=False) \
  .first()

('Santo Antônio do Grama', 97.34121122599704)